# Simple retrieval augmented generation with OpenAI

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
APPLY = True
SAMPLE_COLLECTION_NAME = 'sample_simple_rag'
COLLECTION_NAME = '<var:table_name>' if not APPLY else 'docs'
ID_FIELD = '<var:id_field>' if not APPLY else 'id'
OUTPUT_PREFIX = 'outputs__'
EAGER = True

In [3]:
from superduper import superduper, CFG
import os

db = superduper()

2025-Feb-20 13:50:41.38| INFO     | Duncans-MBP.fritz.box| superduper.misc.importing:13   | Loading plugin: mongodb
2025-Feb-20 13:50:41.44| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:50   | Building Data Layer
2025-Feb-20 13:50:41.44| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:67   | Data Layer built
2025-Feb-20 13:50:41.44| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:86   | Found custom plugins - loading...
2025-Feb-20 13:50:41.74| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:113  | Cluster initialized in 0.30 seconds.
2025-Feb-20 13:50:41.74| INFO     | Duncans-MBP.fritz.box| superduper.base.build:147  | Configuration: 
 +----------------+-----------------------------------+
| Configuration  |               Value               |
+----------------+-----------------------------------+
|  Data Backend  | mongodb://localhost:27017/test_db |
| Artifact Store |   filesystem://./artifact_store   |
+-------------

In [4]:
db.drop(True, True)
db = superduper()

2025-Feb-20 13:50:41.89| INFO     | Duncans-MBP.fritz.box| superduper.misc.importing:13   | Loading plugin: mongodb
2025-Feb-20 13:50:41.89| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:50   | Building Data Layer
2025-Feb-20 13:50:41.90| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:67   | Data Layer built
2025-Feb-20 13:50:41.91| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:113  | Cluster initialized in 0.01 seconds.
2025-Feb-20 13:50:41.91| INFO     | Duncans-MBP.fritz.box| superduper.base.build:147  | Configuration: 
 +----------------+-----------------------------------+
| Configuration  |               Value               |
+----------------+-----------------------------------+
|  Data Backend  | mongodb://localhost:27017/test_db |
| Artifact Store |   filesystem://./artifact_store   |
+----------------+-----------------------------------+


In [5]:
import json
import requests
import io
from superduper import logging


def getter():
    logging.info('Downloading data...')
    response = requests.get('https://superduperdb-public-demo.s3.amazonaws.com/text.json')
    logging.info('Downloading data... (Done)')
    data = json.loads(response.content.decode('utf-8'))
    return [{'x': r} for r in data]

In [6]:
if APPLY:
    data = getter()

2025-Feb-20 13:50:45.45| INFO     | Duncans-MBP.fritz.box| __main__:8    | Downloading data...
2025-Feb-20 13:50:46.26| INFO     | Duncans-MBP.fritz.box| __main__:10   | Downloading data... (Done)


<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [7]:
if APPLY:
    from superduper import Document, Table
    table = Table(COLLECTION_NAME, fields={'x': 'str'})
    db.apply(table, force=True)
    ids = db.execute(db[COLLECTION_NAME].insert(data))

2025-Feb-20 13:50:47.85| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-20 13:50:47.85| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-20 13:50:47.85| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-20 13:50:47.89| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, docs) not found in cache, loading from db
2025-Feb-20 13:50:47.89| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'docs')) from metadata...
2025-Feb-20 13:50:47.89| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:367  | Found new Table:docs:fc8ff3756dc74fd6
2025-Feb-20 13:50:47.89| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:78   | Found these changes and/ or additions that need to be made:
2025-Feb-20 13:50:47.89| INF

Create plugin:

In [8]:
from superduper import Plugin

plugin = Plugin('rag-plugin', path='./rag_plugin.py')

<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [10]:
from superduper import Listener
from rag_plugin import Chunker

upstream_listener = Listener(
    model=Chunker(identifier='chunker'),
    select=db[COLLECTION_NAME],
    key='x',
    identifier='chunker',
    flatten=True,
    upstream=[plugin],
)

In [11]:
if APPLY and EAGER:
    db.apply(upstream_listener, force=True)

2025-Feb-20 13:50:54.08| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-20 13:50:54.08| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-20 13:50:54.08| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-20 13:50:54.09| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-20 13:50:54.09| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-20 13:50:54.09| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-20 13:50:54.10| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Plugin, plugin-rag_plugin_py) not found in cache, loading from db
2025-Fe

## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

<!-- TABS -->
## Build text embedding model

OpenAI:

In [12]:
import os

from superduper_openai import OpenAIEmbedding

openai_embedding = OpenAIEmbedding(
    identifier='text-embedding',
    model='text-embedding-ada-002',
    datatype='vector[float:1536]',
)

## Create vector-index

In [13]:
from superduper import VectorIndex, Listener

vector_index_name = 'vectorindex'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs],
        model=openai_embedding,
        identifier='embeddinglistener',
        upstream=[upstream_listener],
    )
)

In [14]:
if APPLY and EAGER:
    db.apply(vector_index, force=True)

2025-Feb-20 13:51:01.58| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-20 13:51:01.58| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-20 13:51:01.58| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-20 13:51:01.59| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:261  | Found identical Plugin:plugin-rag_plugin_py:rag-plugin
2025-Feb-20 13:51:01.59| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:348  | Found update Plugin:plugin-rag_plugin_py:rag-plugin
2025-Feb-20 13:51:01.59| INFO     | Duncans-MBP.fritz.box| superduper.backends.local.artifacts:131  | Copying file ./rag_plugin.py to ./artifact_store/f74492989c5ac8b5eba5041c05b9927b08d08ffa/rag_plugin.py
2025-Feb-20 13:51:01.59| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:261  | Found ident

vectorindex
├── plugin-rag_plugin_py
│   ├── status: update
│   └── changes
└── chunker
    ├── status: update
    └── changes

2025-Feb-20 13:51:01.61| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:78   | Found these changes and/ or additions that need to be made:
2025-Feb-20 13:51:01.61| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:80   | ----------------------------------------------------------------------------------------------------
2025-Feb-20 13:51:01.61| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:81   | METADATA EVENTS:
2025-Feb-20 13:51:01.61| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:82   | ----------------------------------------------------------------------------------------------------
2025-Feb-20 13:51:01.61| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:89   | [0]: Pluginplugin-rag_plugin_py:rag-plugin: update ~ [1]
2025-Feb-20 13:51:01.61| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:89   | [1]: Listenerchunker:256974701f404ceb: update ~ [4]
2025-Feb-20 13:51:01.61| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:89 

[2025-02-20 13:51:02] httpx INFO HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"


2025-Feb-20 13:51:02.51| INFO     | Duncans-MBP.fritz.box| superduper.components.model:495  | Requesting prediction in db - [text-embedding] with predict_id embeddinglistener__8b71ad9f5f464fee



[2025-02-20 13:51:03] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"                                                                                                                     | 0/4 [00:00<?, ?it/s]
[2025-02-20 13:51:04] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"                                                                                                             | 1/4 [00:01<00:03,  1.32s/it]
[2025-02-20 13:51:06] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"████                                                                                                         | 2/4 [00:03<00:03,  1.63s/it]
[2025-02-20 13:51:07] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"████████████████████████████████████████████████████████▌                                                    | 3/4 [00:04<00:01,  1.43s/it]
100%|███████████

2025-Feb-20 13:51:08.22| INFO     | Duncans-MBP.fritz.box| superduper.components.model:633  | Adding 336 model outputs to `db`
2025-Feb-20 13:51:08.23| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, _outputs__embeddinglistener__8b71ad9f5f464fee) not found in cache, loading from db
2025-Feb-20 13:51:08.23| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', '_outputs__embeddinglistener__8b71ad9f5f464fee')) from metadata...
2025-Feb-20 13:51:08.23| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:_outputs__embeddinglistener__8b71ad9f5f464fee:6c7c73ddac174a5d to cache
2025-Feb-20 13:51:08.72| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Job) not found in cache, loading from db
2025-Feb-20 13:51:08.72| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Job')) from metadata...
2025-Feb-20 13:51:08.72| INFO     | Duncans-

<!-- TABS -->
## Build LLM

In [15]:
from superduper_openai import OpenAIChatCompletion


llm_openai = OpenAIChatCompletion(
    identifier='llm-model',
    model='gpt-3.5-turbo',
    datatype='str',
)

## Answer question with LLM

In [16]:
from superduper import model
from rag_plugin import RAGModel


prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "These snippets are samples from our internal data-repositories, and should be used exclusively and as a matter"
    " of priority to answer the question. Please answer in 20 words or less.\n\n"
    "{context}\n\n"
    "Here's the question: {query}"
)

rag = RAGModel(
    'simple_rag',
    select=db[upstream_listener.outputs].select().like({upstream_listener.outputs: '<var:query>'}, vector_index=vector_index_name, n=5),
    prompt_template=prompt_template,
    key=upstream_listener.outputs,
    llm=llm_openai,
)

In [17]:
if APPLY and EAGER:
    db.apply(rag, force=True)

2025-Feb-20 13:51:24.22| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-20 13:51:24.22| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-20 13:51:24.22| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-20 13:51:24.23| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-20 13:51:24.23| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-20 13:51:24.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-20 13:51:24.25| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (OpenAIChatCompletion, llm-model) not found in cache, loading from db
2025

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [18]:
from superduper import Streamlit, Plugin
from rag_plugin import demo_func

demo = Streamlit('simple-rag-demo', demo_func=demo_func, upstream=[plugin])

In [19]:
from superduper import Application

app = Application(
    'simple-rag-app',
    components=[
        upstream_listener,
        vector_index,
        rag,
        demo,
    ]
)

2025-Feb-20 13:51:28.34| INFO     | Duncans-MBP.fritz.box| superduper.components.application:33   | Resorting components based on topological order.
2025-Feb-20 13:51:28.34| INFO     | Duncans-MBP.fritz.box| superduper.components.application:50   | New order of components: ['Listener:chunker:256974701f404ceb', 'VectorIndex:vectorindex:964215d5b8974b29', 'RAGModel:simple_rag:65eebfe2283e4392', 'Streamlit:simple-rag-demo:0cefeef5b41844e5']


In [20]:
if APPLY:
    db.apply(app, force=True)

2025-Feb-20 13:51:29.18| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-20 13:51:29.18| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-20 13:51:29.18| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-20 13:51:29.20| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:261  | Found identical Plugin:plugin-rag_plugin_py:rag-plugin
2025-Feb-20 13:51:29.20| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:348  | Found update Plugin:plugin-rag_plugin_py:rag-plugin
2025-Feb-20 13:51:29.20| INFO     | Duncans-MBP.fritz.box| superduper.backends.local.artifacts:131  | Copying file ./rag_plugin.py to ./artifact_store/f74492989c5ac8b5eba5041c05b9927b08d08ffa/rag_plugin.py
2025-Feb-20 13:51:29.20| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:261  | Found ident

simple-rag-app
├── plugin-rag_plugin_py
│   ├── status: update
│   └── changes
├── chunker
│   ├── status: update
│   └── changes
└── embeddinglistener
    ├── status: update
    └── changes

2025-Feb-20 13:51:29.25| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:78   | Found these changes and/ or additions that need to be made:
2025-Feb-20 13:51:29.25| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:80   | ----------------------------------------------------------------------------------------------------
2025-Feb-20 13:51:29.25| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:81   | METADATA EVENTS:
2025-Feb-20 13:51:29.25| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:82   | ----------------------------------------------------------------------------------------------------
2025-Feb-20 13:51:29.25| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:89   | [0]: Pluginplugin-rag_plugin_py:rag-plugin: update ~ [3]
2025-Feb-20 13:51:29.25| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:89   | [1]: Listenerchunker:256974701f404ceb: update ~ [4]
2025-Feb-20 13:51:29.25| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:91 

In [21]:
db.show()

[{'component': 'Application', 'identifier': 'simple-rag-app'},
 {'component': 'Chunker', 'identifier': 'chunker'},
 {'component': 'Listener', 'identifier': 'chunker'},
 {'component': 'Listener', 'identifier': 'embeddinglistener'},
 {'component': 'OpenAIChatCompletion', 'identifier': 'llm-model'},
 {'component': 'OpenAIEmbedding', 'identifier': 'text-embedding'},
 {'component': 'Plugin', 'identifier': 'plugin-rag_plugin_py'},
 {'component': 'RAGModel', 'identifier': 'simple_rag'},
 {'component': 'Streamlit', 'identifier': 'simple-rag-demo'},
 {'component': 'Table', 'identifier': 'Application'},
 {'component': 'Table', 'identifier': 'Chunker'},
 {'component': 'Table', 'identifier': 'Listener'},
 {'component': 'Table', 'identifier': 'OpenAIChatCompletion'},
 {'component': 'Table', 'identifier': 'OpenAIEmbedding'},
 {'component': 'Table', 'identifier': 'Plugin'},
 {'component': 'Table', 'identifier': 'RAGModel'},
 {'component': 'Table', 'identifier': 'Streamlit'},
 {'component': 'Table', '

In [ ]:
if APPLY:
    rag = db.load('RAGModel', 'simple_rag')
    print(rag.predict('Tell me about the project'))

In [ ]:
rag.predict('Tell me about vector-search')

You can now load the model elsewhere and make predictions using the following command.

## Create template

In [ ]:
from superduper import Template, Table, Schema
from superduper.components.dataset import RemoteData

template = Template(
    'simple_rag',
    template=app,
    substitutions={
        COLLECTION_NAME: 'table_name',
        'text-embedding-ada-002': 'embedding_model',
        'gpt-3.5-turbo': 'llm_model',
    },
    template_variables=['table_name', 'id_field', 'embedding_model', 'llm_model'],
    default_tables=[
        Table(
            'sample_simple_rag',
            fields={'x': 'str'},
            data=RemoteData(
                'superduper-docs',
                getter=getter,
            )
        ),
    ],
    types={
        'id_field': {
            'type': 'str',
            'default': '_id',
        },
        'embedding_model': {
            'type': 'str',
            'default': 'text-embedding-ada-002',
            'choices': ['text-embedding-ada-002', 'nomic-embed-text:latest'],
        },
        'llm_model': {
            'type': 'str',
            'default': 'gpt-3.5-turbo',
            'choices': ['gpt-3.5-turbo', 'gpt-4-turbo', 'llama3.1:8b']
        },
        'table_name': {
            'type': 'str',
            'default': SAMPLE_COLLECTION_NAME,
        }
    },
    schema={
        "id_field": "id_field",
        "embedding_model": "embedding_model",
        "llm_model": "llm_model",
        "table_name": "table_name",
    },
    db=db
)

In [ ]:
template.export('.')